<a href="https://colab.research.google.com/github/mahsaghaderan99/MLP/blob/backpropagation/mlp123.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Deep MLP project

#Part1

##Import

In [1]:
import tensorflow as tf
from keras.utils.np_utils import to_categorical  
from os import path, getcwd, chdir
import numpy as np

##Layer

In [137]:
class Layer:
  def __init__(self,node_number,activation):
    
    self.node_num = node_number
    self.delta = np.empty(node_number)
    self.vj = np.empty(node_number)
    self.outj = np.empty(node_number)
    self.b = np.zeros((self.node_num, 1)) 
    self.activation = self._sigmoid if activation == 'sigmoid' else self._relu
    self.d_activation = self._d_sigmoid if activation == 'sigmoid' else self._d_relu

  def define_weights(self,input_shape):
    self.weights = np.random.uniform(low=0, high=1, size=(self.node_num,input_shape))

  def _sigmoid(self,x):
    return np.power(1+np.exp(-0.5*x),-1)
    
  def _d_sigmoid(self,x):
    return self._sigmoid(x)*(1-self._sigmoid(x))

  def _relu(self,x):
    return x*(x>0)

  def _d_relu(self,x):
    return 1.*(x>0)
   

##Model

In [203]:
class Model:
  def __init__(self,learning_rate,epochs):
    self.layers = np.empty(0,dtype=Layer)
    self.lr = learning_rate
    self.epochs = epochs

  def Dense(self,node_number,activation_function):
    self.layers = np.append(self.layers,Layer(node_number,activation_function))

  def Train(self,training,training_y):
    try:
      self.training = training.reshape(training.shape[0],training.shape[1]*training.shape[2])
    except:
      self.training = np.expand_dims(training,1)
    self.desiered_labels = training_y
    self._define_weights()
    while self.epochs>0:
      print(self.epochs)
      self.epochs -=1
      for tr in range(0,self.training.shape[0]):
        yhat = self._forward_propagation(tr)
        self._update_weights(yhat,tr)

  def _define_weights(self):
    self.layers[0].define_weights(self.training[0].shape[0])
    for i in range(1,self.layers.shape[0]):
      self.layers[i].define_weights(self.layers[i-1].node_num)

  def _forward_propagation(self,tr):
    yhat = self._layer_output(self.training[tr] ,0)
    for i in range(1,self.layers.shape[0]):
      yhat = self._layer_output(yhat ,i)
    # print( yhat)
    return yhat

  def _layer_output(self,x,i):
    yhat = np.dot(self.layers[i].weights,x)
    if i!= self.layers.shape[0]:
      yhat = self.layers[i].activation(yhat)
    self.layers[i].yj = yhat
    return yhat

  def _update_weights(self,yhat,tr):
    fiprim = self.layers[-1].d_activation(self.layers[-1-1].yj)
    fiprim = np.expand_dims(fiprim,1).T
    self.layers[-1].delta  = (fiprim * (self.desiered_labels[tr]-self.layers[-1].yj))
    # print(self.desiered_labels[tr]-self.layers[-1-1].yj)
    for i in range(len(self.layers)-2,-1,-1):
      self._delta(i,tr)
    for i in range(len(self.layers)-1,-1,-1):
      yj =np.expand_dims(self.layers[i].yj,1)
      delta = self.layers[i].delta
      deltaw = self.lr * delta * yj
      # print(delta)
      self.layers[i].weights = self.layers[i].weights + deltaw

  def _delta(self,i,tr):
    if i == 0: 
      input = self.training[tr]
    else:
      input = self.layers[i-1].yj
    fiprim =  self.layers[i-1].d_activation(input)
    fiprim = np.expand_dims(fiprim,1)
    sum_delta_w = np.sum((self.layers[i+1].weights*self.layers[i+1].delta).T,1)
    self.layers[i].delta =  (fiprim * sum_delta_w).T
  
  def Predict(self,test_images):
    try:
      testings = test_images.reshape(test_images.shape[0],test_images.shape[1]*test_images.shape[2])
    except:
      testings = test_images
    predicts = []
    for t in range(0,testings.shape[0]):
      yhat = self._layer_output(testings[t] ,0)
      for i in range(1,self.layers.shape[0]):
        yhat = self._layer_output(yhat ,i)
      predicts.append(yhat[0][0])
    return np.array(predicts)
    
  def Evaluate(self,x,y):
    predictions = self.Predict(x)
    return np.sum(np.power(predictions-y,2))/2, np.sum(np.abs(predictions-y))/y.shape[0]

#Part2

In [185]:
x = np.linspace(0,1,200,dtype=np.float16)
np.random.shuffle(x)


In [204]:
a = Model(0.01,20)
a.Dense(4,'relu')
a.Dense(1,'relu')
a.Train(x[10:40],x[10:40])
# a.Test(x[10:40],x[10:40])
print(a.Evaluate(x[10:40], x[10:40]))
p = a.Predict(x[10:40])
print(len(p))
for i in range(len(p)):
  print(p[i],x[i+10])
# # to_categorical(training_y, num_classes)

20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
(1.0297018896930208e-08, 2.2203289229272975e-05)
30
0.7739610099214337 0.774
0.05526984972270175 0.05527
0.276379767580777 0.2764
0.38698050496071684 0.387
0.24122191728781592 0.2412
0.26636954631680887 0.2664
0.5478765003986427 0.548
0.2712525810797201 0.2712
0.8594141182723807 0.8594
0.5781513159286924 0.578
0.9751420421533774 0.975
0.1759113273338779 0.1759
0.371843097195692 0.3718
0.346695468166699 0.3467
0.6333296087495895 0.6333
0.477316648074575 0.4773
0.41212813398970977 0.412
0.05026473909071772 0.05026
0.7388031596284726 0.739
0.09546332961491502 0.09546
0.35670568943066705 0.3567
0.9800250769162886 0.98
0.869180187798203 0.869
0.6631161208033483 0.663
0.17090621670189388 0.1709
0.30152739660976996 0.3015
0.9497502613862389 0.9497
0.8091188602143946 0.809
0.010048369973053313 0.01005
0.989791146442111 0.9897


In [ ]:
#show classificaion and error
import matplotlib.pyplot as plt

nerr =np.arange(0,a.err.shape[0])
print(a.err.shape[0]) 
plt.plot(nerr,a.err,'r')
plt.show()